In [101]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [87]:
voter_profile_df = pd.read_csv("voter_profile/data.csv")
voter_profile_df.drop(["male", "female"], axis=1, inplace=True)
df = voter_profile_df.set_index(["city", "district", "age"]).unstack().reset_index()
df.columns = [f"{i}_{j}" for i, j in df.columns]
df.rename(columns={i: i.replace("_", "") for i in df.columns}, inplace=True)
df.rename(columns={i: i.replace("all", "") for i in df.columns}, inplace=True)

x_cols = [i for i in df.columns if i not in ["city", "district"]]
df["total"] = df[x_cols].sum(axis=1)

for col in x_cols:
    df[f"{col}_ratio"] = df[col] / df["total"]

df.rename(columns={i: f"{i}_count" for i in x_cols}, inplace=True)
df.drop("total", axis=1, inplace=True)
df

,city,district,18-24_count,25-29_count,30-34_count,35-39_count,40-44_count,45-49_count,50-54_count,55-59_count,...,30-34_ratio,35-39_ratio,40-44_ratio,45-49_ratio,50-54_ratio,55-59_ratio,60-64_ratio,65-69_ratio,70-74_ratio,75+_ratio
0,Adana,Aladağ,1867,1053,1086,1145,1000,927,888,942,...,0.090364,0.095274,0.083209,0.077134,0.073889,0.078382,0.074305,0.056665,0.047678,0.080130
1,Adana,Ceyhan,18197,11478,11098,11287,9715,9194,9465,8796,...,0.100240,0.101947,0.087749,0.083043,0.085491,0.079448,0.067760,0.047979,0.031125,0.047185
2,Adana,Feke,1725,966,964,1041,1038,1175,1014,1040,...,0.076727,0.082856,0.082617,0.093521,0.080707,0.082776,0.078160,0.062639,0.053009,0.092805
3,Adana,Karaisalı,2408,1227,1171,1261,1350,1490,1430,1339,...,0.073000,0.078611,0.084159,0.092887,0.089147,0.083474,0.073312,0.059473,0.047690,0.091640
4,Adana,Karataş,2411,1505,1512,1559,1502,1398,1410,1351,...,0.093616,0.096527,0.092997,0.086558,0.087301,0.083648,0.074175,0.055105,0.036406,0.051204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967,Şırnak,Güçlükonak,1907,1241,680,555,405,290,285,228,...,0.106868,0.087223,0.063649,0.045576,0.044790,0.035832,0.038975,0.030017,0.017759,0.034575
968,Şırnak,Silopi,18898,12864,8835,7514,5251,4289,3296,2025,...,0.129430,0.110077,0.076925,0.062832,0.048285,0.029666,0.025007,0.018766,0.011559,0.022150
969,Şırnak,Uludere,6341,4203,2222,2647,1858,1348,1229,845,...,0.095954,0.114307,0.080235,0.058211,0.053073,0.036490,0.031006,0.026558,0.018871,0.029969
970,Şırnak,İdil,11160,6358,4253,3826,3003,2384,1670,1280,...,0.112252,0.100982,0.079260,0.062922,0.044077,0.033784,0.032438,0.026050,0.016971,0.028901


In [88]:
election_results_df = pd.read_csv("election_results/data.csv")
election_results_df

election_results_df.rename(
    columns={
        "Kayıtlı seçmen sayısı": "voter_count",
        "CHP": "chp_vote_count",
        "AK PARTİ": "akp_vote_count",
        "Geçerli oy sayısı": "total_voter_count",
    },
    inplace=True,
)

election_results_df["akp_vote_ratio"] = (
    election_results_df["akp_vote_count"] / election_results_df["total_voter_count"]
)
election_results_df["chp_vote_ratio"] = (
    election_results_df["chp_vote_count"] / election_results_df["total_voter_count"]
)

election_results_df = election_results_df[
    [
        "city",
        "district",
        "akp_vote_count",
        "chp_vote_count",
        "akp_vote_ratio",
        "chp_vote_ratio",
    ]
]
election_results_df

,city,district,akp_vote_count,chp_vote_count,akp_vote_ratio,chp_vote_ratio
0,Kütahya,Altıntaş,6152,1218,0.573881,0.113619
1,Kütahya,Domaniç,6073,1344,0.567676,0.125631
2,Kütahya,Emet,8363,1344,0.591736,0.095097
3,Kütahya,Gediz,19464,3823,0.567861,0.111536
4,Kütahya,Kütahya Merkez,79048,23636,0.486785,0.145553
...,...,...,...,...,...,...
977,Ankara,Beypazarı,14822,3517,0.463492,0.109978
978,Ankara,Nallıhan,8890,3661,0.455524,0.187590
979,Ankara,Yenimahalle,147343,131397,0.341954,0.304946
980,Ankara,Sincan,167607,42449,0.520839,0.131910


In [89]:
df = df.merge(election_results_df, on=["city", "district"], how="left")
df

,city,district,18-24_count,25-29_count,30-34_count,35-39_count,40-44_count,45-49_count,50-54_count,55-59_count,...,50-54_ratio,55-59_ratio,60-64_ratio,65-69_ratio,70-74_ratio,75+_ratio,akp_vote_count,chp_vote_count,akp_vote_ratio,chp_vote_ratio
0,Adana,Aladağ,1867,1053,1086,1145,1000,927,888,942,...,0.073889,0.078382,0.074305,0.056665,0.047678,0.080130,5092,1557,0.484629,0.148187
1,Adana,Ceyhan,18197,11478,11098,11287,9715,9194,9465,8796,...,0.085491,0.079448,0.067760,0.047979,0.031125,0.047185,28443,21430,0.305104,0.229876
2,Adana,Feke,1725,966,964,1041,1038,1175,1014,1040,...,0.080707,0.082776,0.078160,0.062639,0.053009,0.092805,5818,1326,0.547216,0.124718
3,Adana,Karaisalı,2408,1227,1171,1261,1350,1490,1430,1339,...,0.089147,0.083474,0.073312,0.059473,0.047690,0.091640,7879,1292,0.557095,0.091353
4,Adana,Karataş,2411,1505,1512,1559,1502,1398,1410,1351,...,0.087301,0.083648,0.074175,0.055105,0.036406,0.051204,3457,5233,0.247583,0.374776
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967,Şırnak,Güçlükonak,1907,1241,680,555,405,290,285,228,...,0.044790,0.035832,0.038975,0.030017,0.017759,0.034575,1185,107,0.222076,0.020052
968,Şırnak,Silopi,18898,12864,8835,7514,5251,4289,3296,2025,...,0.048285,0.029666,0.025007,0.018766,0.011559,0.022150,9446,1194,0.157741,0.019939
969,Şırnak,Uludere,6341,4203,2222,2647,1858,1348,1229,845,...,0.053073,0.036490,0.031006,0.026558,0.018871,0.029969,3479,757,0.188727,0.041065
970,Şırnak,İdil,11160,6358,4253,3826,3003,2384,1670,1280,...,0.044077,0.033784,0.032438,0.026050,0.016971,0.028901,5330,497,0.177584,0.016559


## Correlations


In [103]:
count_cols = [
    i
    for i in df.columns
    if "count" in i and i not in ["chp_vote_count", "akp_vote_count"]
]

ratio_cols = [
    i
    for i in df.columns
    if "ratio" in i and i not in ["chp_vote_ratio", "akp_vote_ratio"]
]

parties = ["akp", "chp"]
col_groups = {"count": count_cols, "ratio": ratio_cols}

In [106]:
for group in col_groups:
    cols = col_groups[group]
    for col in cols:
        print(col)
        print(f"AKP {group} Correlation: ", round(df[col].corr(df["akp_vote_count"]), 2))
        print(f"CHP {group} Correlation: ", round(df[col].corr(df["chp_vote_count"]), 2))

18-24_count
AKP count Correlation:  0.95
CHP count Correlation:  0.77


25-29_count
AKP count Correlation:  0.94
CHP count Correlation:  0.81


30-34_count
AKP count Correlation:  0.95
CHP count Correlation:  0.83


35-39_count
AKP count Correlation:  0.95
CHP count Correlation:  0.85


40-44_count
AKP count Correlation:  0.95
CHP count Correlation:  0.88


45-49_count
AKP count Correlation:  0.94
CHP count Correlation:  0.89


50-54_count
AKP count Correlation:  0.93
CHP count Correlation:  0.9


55-59_count
AKP count Correlation:  0.91
CHP count Correlation:  0.92


60-64_count
AKP count Correlation:  0.89
CHP count Correlation:  0.93


65-69_count
AKP count Correlation:  0.86
CHP count Correlation:  0.93


70-74_count
AKP count Correlation:  0.83
CHP count Correlation:  0.92


75+_count
AKP count Correlation:  0.78
CHP count Correlation:  0.9


18-24_ratio
AKP ratio Correlation:  0.1
CHP ratio Correlation:  -0.09


25-29_ratio
AKP ratio Correlation:  0.28
CHP ratio Correlation:  0.1

## Linear Regression


In [109]:
for party in parties:
    for group in col_groups:
        print(party, group)
        cols = col_groups[group]
        reg = LinearRegression(positive=True).fit(df[cols], df[f"{party}_vote_{group}"])
        print(reg.score(df[cols], df[f"{party}_vote_{group}"]))
        for i, j in zip(cols, reg.coef_):
            print(i, j)
        print(reg.intercept_)
        print("\n")

akp count
0.9125888480268445
18-24_count 1.1472006789790794
25-29_count 0.0
30-34_count 0.39705474026387144
35-39_count 0.0
40-44_count 0.0
45-49_count 1.404776661291795
50-54_count 0.0
55-59_count 0.0
60-64_count 0.0
65-69_count 0.0
70-74_count 0.0
75+_count 0.0
893.7143552544694


akp ratio
0.22954947951191396
18-24_ratio 6.418710148461289
25-29_ratio 0.0
30-34_ratio 10.485019545663851
35-39_ratio 1.8427283162216266
40-44_ratio 4.416006593760451
45-49_ratio 4.622834302884565
50-54_ratio 5.401939385549824
55-59_ratio 2.576029318516979
60-64_ratio 6.839873421670655
65-69_ratio 4.716096846632163
70-74_ratio 5.1722636284094765
75+_ratio 5.352531498152721
-4.399573782869544


chp count
0.8619600492620612
18-24_count 0.0
25-29_count 0.0
30-34_count 0.0
35-39_count 0.0
40-44_count 0.0
45-49_count 0.0
50-54_count 0.0
55-59_count 0.0
60-64_count 3.3393965020646035
65-69_count 0.0
70-74_count 0.0
75+_count 1.4663821018935583
-4073.4800242700258


chp ratio
0.4859908627089107
18-24_ratio 0.8804